In [40]:
import sys
sys.path.append("..")
import pandas as pd
import pickle

## Timings

In [25]:
timing_stats = pd.concat(
    [pd.read_csv(f"../results/{dataset}_gen_db_stats.csv", index_col=None).assign(dataset=dataset)
    for dataset in ["fico", "german", "givemecredit"]]
)

In [26]:
timing_stats.groupby("dataset").agg(["mean", "max"])

Unnamed: 0           n_points         complete            time  \
                   mean   max         mean     max     mean   max      mean   
dataset                                                                       
fico             2807.5  5615  9661.209758  123904      1.0  True  1.473479   
german            498.0   996   106.808425     432      1.0  True  0.163755   
givemecredit      747.0  1494    58.763211     432      1.0  True  0.150193   

                        
                   max  
dataset                 
fico          7.088939  
german        0.386051  
givemecredit  0.246207

In [45]:
from time import time

In [46]:
start = time()
fico_xgb_data["data"]["model"].predict(fico_data["data"].X)
time() - start

0.02065277099609375

In [47]:
with open("../data/fico_complex_nD.data", "rb") as f:
    fico_data = pickle.load(f)

In [31]:
with open("../results/fico_complex_nD_xgb_processed.model", "rb") as f:
    fico_xgb_data = pickle.load(f)

In [48]:
with open("../results/fico_complex_nD_xgb_dice.audit", "rb") as f:
    fico_xgb_audit = pickle.load(f)

## Dice loopholes that modify several features

In [162]:
num_action_threshold = 5

In [141]:
audit_df = pd.DataFrame(fico_xgb_audit["data"].values())

In [166]:
num_actions = (
    audit_df
    .query("actionable == False and recourse_exists == True and orig_prediction == False")
    .apply(lambda val: abs(val.a.sum()), axis=1)
    .dropna()
)

multiple_change_indices = num_actions[num_actions >= 12].index
(num_actions >= 5).mean()

0.2689199118295371

In [167]:
target_idx = multiple_change_indices[0]
target_idx

1428

In [183]:
print(pd.DataFrame(
    np.vstack([
        fico_data["data"].X_df.loc[target_idx].values,
        audit_df.loc[target_idx].a
    ]).T,
    index=fico_data["data"].X_df.columns,
    columns=["x", "a"]
))

                                           x    a
ExternalRiskEstimate_geq_40              1.0  0.0
ExternalRiskEstimate_geq_50              1.0  0.0
ExternalRiskEstimate_geq_60              1.0  0.0
ExternalRiskEstimate_geq_70              0.0  0.0
ExternalRiskEstimate_geq_80              0.0  0.0
YearsOfAccountHistory                   10.0  0.0
AvgYearsInFile_geq_3                     1.0  0.0
AvgYearsInFile_geq_5                     1.0  0.0
AvgYearsInFile_geq_7                     0.0  0.0
MostRecentTradeWithinLastYear            1.0 -1.0
MostRecentTradeWithinLast2Years          1.0  0.0
AnyDerogatoryComment                     0.0  0.0
AnyTrade120DaysDelq                      0.0  0.0
AnyTrade90DaysDelq                       0.0  0.0
AnyTrade60DaysDelq                       0.0  0.0
AnyTrade30DaysDelq                       0.0  0.0
NoDelqEver                               1.0  0.0
YearsSinceLastDelqTrade_leq_1            1.0  0.0
YearsSinceLastDelqTrade_leq_3            1.0  0.0
